#Tutorial 5 - Options Intro

Please complete this tutorial to get an overview of options and an implementation of SMDP Q-Learning and Intra-Option Q-Learning.


### References:

 [Recent Advances in Hierarchical Reinforcement
Learning](https://people.cs.umass.edu/~mahadeva/papers/hrl.pdf) is a strong recommendation for topics in HRL that was covered in class. Watch Prof. Ravi's lectures on moodle or nptel for further understanding the core concepts. Contact the TAs for further resources if needed. 


In [1]:
'''
A bunch of imports, you don't have to worry about these
'''

import numpy as np
import random
import gym
from gym.wrappers import Monitor
import glob
import io
import matplotlib.pyplot as plt
from IPython.display import HTML

In [2]:
'''
The environment used here is extremely similar to the openai gym ones.
At first glance it might look slightly different. 
The usual commands we use for our experiments are added to this cell to aid you
work using this environment.
'''

#Setting up the environment
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv
env = CliffWalkingEnv()

env.reset()

#Current State
print(env.s)

# 4x12 grid = 48 states
print ("Number of states:", env.nS)

# Primitive Actions
action = ["up", "right", "down", "left"]
#correspond to [0,1,2,3] that's actually passed to the environment

# either go left, up, down or right
print ("Number of actions that an agent can take:", env.nA)

# Example Transitions
rnd_action = random.randint(0, 3)
print ("Action taken:", action[rnd_action])
next_state, reward, is_terminal, t_prob = env.step(rnd_action)
print ("Transition probability:", t_prob)
print ("Next state:", next_state)
print ("Reward recieved:", reward)
print ("Terminal state:", is_terminal)
env.render()

36
Number of states: 48
Number of actions that an agent can take: 4
Action taken: down
Transition probability: {'prob': 1.0}
Next state: 36
Reward recieved: -1
Terminal state: False
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
o  o  o  o  o  o  o  o  o  o  o  o
x  C  C  C  C  C  C  C  C  C  C  T



#### Options
We custom define very simple options here. They might not be the logical options for this settings deliberately chosen to visualise the Q Table better.


In [3]:
# We are defining two more options here
# Option 1 ["Away"] - > Away from Cliff (ie keep going up)
# Option 2 ["Close"] - > Close to Cliff (ie keep going down) 

def Away(env,state):
    
    optdone = False
    optact = 0
    
    if (int(state/12) == 0):
        optdone = True
    
    return [optact,optdone]
    
def Close(env,state):
    
    optdone = False
    optact = 2
    
    if (int(state/12) == 2):
        optdone = True

    if (int(state/12) == 3):
        optdone = True
    
    return [optact,optdone]
    
    
'''
Now the new action space will contain
Primitive Actions: ["up", "right", "down", "left"]
Options: ["Away","Close"]
Total Actions :["up", "right", "down", "left", "Away", "Close"]
Corresponding to [0,1,2,3,4,5]
'''

'\nNow the new action space will contain\nPrimitive Actions: ["up", "right", "down", "left"]\nOptions: ["Away","Close"]\nTotal Actions :["up", "right", "down", "left", "Away", "Close"]\nCorresponding to [0,1,2,3,4,5]\n'

# Task 1
Complete the code cell below


In [4]:
#Q-Table: (States x Actions) === (env.ns(48) x total actions(6))
q_values_SMDP = np.zeros((48,6))

#Update_Frequency Data structure? Check TODO 4

# TODO: epsilon-greedy action selection function
def egreedy_policy(q_values,state,epsilon):
    rnd = np.random.random_sample()
    if(rnd < epsilon):
        return np.random.randint(q_values.shape[-1]) # total actions are last dimension of q values, ie. 6
    else:
        return np.argmax(q_values[state])

# Task 2
Below is an incomplete code cell with the flow of SMDP Q-Learning. Complete the cell and train the agent using SMDP Q-Learning algorithm.
Keep the **final Q-table** and **Update Frequency** table handy (You'll need it in TODO 4)

In [5]:
#### SMDP Q-Learning 

# Add parameters you might need here
gamma = 0.99
alpha = 0.6
update_frequency_smdp = np.zeros(48)
rewards = []

# Iterate over 1000 episodes
for epi in range(1000):
    state = env.reset()    
    done = False

    total_reward = 0

    # While episode is not over
    while not done:
        
        # Choose action        
        action = egreedy_policy(q_values_SMDP, state, epsilon=0.01)
        
        # Checking if primitive action
        if action < 4:
            # Perform regular Q-Learning update for state-action pair
            next_state, reward, done,_ = env.step(action)
            q_values_SMDP[state, action] += alpha * (reward + gamma*(q_values_SMDP[next_state].max()) - q_values_SMDP[state, action])
            update_frequency_smdp[state] +=1
            total_reward += reward
        
        # Checking if action chosen is an option
        reward_bar = 0
        if action == 4: # action => Away option
            
            optdone = False
            ini_state = state
            count = 0
            while (optdone == False):
                
                # Think about what this function might do?
                optact,optdone = Away(env,state) 
                next_state, reward, done,_ = env.step(optact)
                total_reward += reward
                
                # Is this formulation right? What is this term?
                reward_bar = gamma*reward_bar + reward
                count += 1
            
                # Complete SMDP Q-Learning Update
                # Remember SMDP Updates. When & What do you update? 
                if(optdone):
                    q_values_SMDP[ini_state, action] += alpha * (reward_bar + (gamma ** count)*(q_values_SMDP[next_state].max()) - q_values_SMDP[ini_state, action])
                    update_frequency_smdp[ini_state] +=1
                
                state = next_state
           
        if action == 5: # action => Close option
            
            optdone = False
            ini_state = state
            count = 0
            while (optdone == False):
                
                # Think about what this function might do?
                optact,optdone = Close(env,state) 
                next_state, reward, done,_ = env.step(optact)
                total_reward += reward
                
                # Is this formulation right? What is this term?
                reward_bar = gamma*reward_bar + reward
                count += 1
            
                # Complete SMDP Q-Learning Update
                # Remember SMDP Updates. When & What do you update? 
                if(optdone):
                    q_values_SMDP[ini_state, action] += alpha * (reward_bar + (gamma ** count)*(q_values_SMDP[next_state].max()) - q_values_SMDP[ini_state, action])
                    update_frequency_smdp[ini_state] +=1
                
                state = next_state
    print("\rEpisode ", epi+1, " Reward ", total_reward, end='')
    rewards.append(total_reward)
print("")


Episode  1000  Reward  -328


# Task 3
Using the same options and the SMDP code, implement Intra Option Q-Learning (In the code cell below). You *might not* always have to search through options to find the options with similar policies, think about it. Keep the **final Q-table** and **Update Frequency** table handy (You'll need it in TODO 4)



In [6]:
#### Intra-Option Q-Learning 

# Add parameters you might need here
#Q-Table: (States x Actions) === (env.ns(48) x total actions(6))
q_values_intraq = np.zeros((48,6))
gamma = 0.99
alpha = 0.6
update_frequency_intraq = np.zeros(48)
rewards = []

# Iterate over 1000 episodes
for epi in range(1000):
    state = env.reset()    
    done = False

    total_reward = 0

    # While episode is not over
    while not done:
        
        # Choose action        
        action = egreedy_policy(q_values_intraq, state, epsilon=0.01)
        
        # Checking if primitive action
        if action < 4:
            # Perform regular Q-Learning update for state-action pair
            next_state, reward, done,_ = env.step(action)
            q_values_intraq[state, action] += alpha * (reward + gamma*(q_values_intraq[next_state].max()) - q_values_intraq[state, action])
            update_frequency_intraq[state] +=1
            total_reward += reward
        
        # Checking if action chosen is an option
        if action == 4: # action => Away option
            
            optdone = False
            while (optdone == False):
                
                # Think about what this function might do?
                optact,optdone = Away(env,state) 
                next_state, reward, done,_ = env.step(optact)
                total_reward += reward
            
                # Complete IntraQ Q Learning update
                q_values_intraq[state, optact] += alpha * (reward + gamma*(q_values_intraq[next_state].max()) - q_values_intraq[state, optact])
                if(optdone):
                    q_values_intraq[state, action] += alpha * (reward + gamma*(q_values_intraq[next_state].max()) - q_values_intraq[state, action])
                else:
                    q_values_intraq[state, action] += alpha * (reward + gamma*(q_values_intraq[next_state,action]) - q_values_intraq[state, action])
                update_frequency_intraq[state] += 1
                
                state = next_state
           
        if action == 5: # action => Close option
            
            optdone = False
            while (optdone == False):
                
                # Think about what this function might do?
                optact,optdone = Away(env,state) 
                next_state, reward, done,_ = env.step(optact)
                total_reward += reward
            
                # Complete IntraQ Q Learning update
                q_values_intraq[state, optact] += alpha * (reward + gamma*(q_values_intraq[next_state].max()) - q_values_intraq[state, optact])
                if(optdone):
                    q_values_intraq[state, action] += alpha * (reward + gamma*(q_values_intraq[next_state].max()) - q_values_intraq[state, action])
                else:
                    q_values_intraq[state, action] += alpha * (reward + gamma*(q_values_intraq[next_state,action]) - q_values_intraq[state, action])
                update_frequency_intraq[state] += 1
                
                state = next_state
    print("\rEpisode ", epi+1, " Reward ", total_reward, end='')
    rewards.append(total_reward)
print("")

Episode  1000  Reward  -545


# Task 4
Compare the two Q-Tables and Update Frequencies and provide comments.

In [7]:
# Use this cell for Task 4 Code
def plot_q(q_values):
  # act = []
  for i in range(4):
    row = []
    for j in range(12):
      s = 4*i + j
      values = q_values[s]
      astar = np.argmax(values)
      if(astar == 0):
        row.append('-U')
      elif(astar == 1):
        row.append('-R')
      elif(astar == 2):
        row.append('-D')
      elif(astar == 3):
        row.append('-L')
      elif(astar == 4):
        row.append('OA')
      elif(astar == 5):
        row.append('OC')
    print(row)

print("POLICY FOR SMDP")
plot_q(q_values_SMDP)

print("POLICY FOR INTRA OPTION")
plot_q(q_values_intraq)

print("Q VALUES SMDP")
print(q_values_SMDP)
print("Q VALUES INTRA OPTION")
print(q_values_intraq)
print("UPDATE FREQUENCY SMDP")
print(update_frequency_smdp)
print("UPDATE FREQUENCY INTRA OPTION")
print(update_frequency_intraq)

POLICY FOR SMDP
['-R', '-R', '-L', '-R', '-D', '-R', '-R', '-D', '-R', 'OA', '-R', 'OC']
['-D', '-R', '-R', '-D', '-R', 'OA', '-R', 'OC', '-U', 'OA', '-R', 'OC']
['-R', 'OA', '-R', 'OC', '-U', 'OA', '-R', 'OC', '-U', '-U', '-U', '-U']
['-U', 'OA', '-R', 'OC', '-U', '-U', '-U', '-U', '-U', '-U', 'OC', 'OC']
POLICY FOR INTRA OPTION
['-R', '-R', '-R', '-R', '-L', '-R', 'OA', '-R', '-R', '-R', '-R', '-D']
['-L', '-R', 'OA', '-R', '-R', '-R', '-R', '-D', '-R', '-U', '-U', '-R']
['-R', '-R', '-R', '-D', '-R', '-U', '-U', '-R', '-R', '-R', '-R', 'OC']
['-R', '-U', '-U', '-R', '-R', '-R', '-R', 'OC', '-R', '-U', '-R', '-D']
Q VALUES SMDP
[[  -4.78922951   -3.940399    -12.65797985   -4.94104057   -4.62599965
    -7.04855492]
 [  -5.02903269   -2.9952025    -5.32044538  -12.80774137   -4.11509961
   -68.07265895]
 [ -28.09716271  -21.31157379  -69.94756204   -2.39719026   -9.87557177
   -46.26082065]
 [  -4.60254709   -2.9701      -24.0566821    -3.18671626   -4.20988567
   -13.27162582]
 [  -1

We can clearly see the differences between the SMDP and INTRA OPTION update frequencies.

Intra option has significantly larger frequencies than smdp models. By the way, by update frequency, I mean the frequency with which each state was updated. 

This leads to two things. SMDP is better because it doesn't have to update again and again, but it also does not update multiple states while it is executing an option. In which regard, Intra option is better.

We can also see this through the Q values. Intra option q values are better adjusted, but SMDP has better estimated the policy at only but a few states.
